In [ ]:
import sys
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from tensorflow import keras

parent_dir = os.path.abspath("../")
sys.path.append(parent_dir)

from predict import _predict_board as pre
from predict import _extra_number as ex
#from preprocess import _extract_num as exnum

import NumberPlace
#from preprocess import _extract_num as exnum


 #日本語対応フォントを指定する
plt.rcParams['font.family'] = 'MS Gothic'

img_path = "../pictures/output1.png" #コーナー検出済みの画像をpicturesフォルダから読み込む

if not os.path.exists(img_path): #画像がない場合
    print(f"画像が見つかりません: {img_path}")
else:
    size = 9
    num_img = cv2.imread(img_path)

    imgN = num_img.copy()
    imgP = num_img.copy()

    split = pre.cells2(num_img, size=size)   #盤面を各マスに分割して切り出す
    #print(split.shape) #81 100 100 1

    processed_split = []

    for i in range(len(split)):   #各マスのノイズを除去、数字を中央に寄せる
        cop = split[i].copy()
        cop = (cop.squeeze() * 255).astype(np.uint8)
        cop = cv2.resize(cop,(28,28))

        cop = ex.extract_center_object_distant(cop, radius=7,distance_threshold=2)
        cop = ex.center_object_by_centroid(cop)
        cop = cop.reshape(28,28,1)
        processed_split.append(cop)



    processed_split = np.array(processed_split)
    #print(processed_split.shape) #81 28 28 1


    model = keras.models.load_model("model_font2.keras")
    pred = model.predict(processed_split)                  #モデルによる数字の推定

    #ノイズに合わせて推定結果を書き換える #この処理は必要！
    for i in range(len(split)):

        center = split[i][25:74, 25:74, :]  # 処理前の画像の中央の49×49を取り出す

        # 黒と判定するしきい値（2値化されているため黒は0）
        black_threshold = 0

        # 黒の割合のしきい値
        ratio_b = 0.90

        # 黒ピクセルの割合を計算(中央の黒の割合が90%以上ならば推定結果を0に書き換える)

        black_ratio = np.mean(center <= black_threshold)
        if black_ratio >= ratio_b:
           pred[i] = [1,0,0,0,0,0,0,0,0,0]    # 0に書き換える（one-hot)

           
    pred_class = np.argmax(pred, axis=1)  #推測結果
    labels_grid = pred_class.reshape(size,size)

    print("\n誤認識の箇所がある場合は修正を行ってください")

        #推測結果を元画像の上から描画
    img = pre.draw_labels_on_image(imgP, labels=labels_grid, size=9)

    plt.figure(figsize=(12, 6))  
    plt.title("Detected_figure\n")
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

    # 画像の外に行と列を表示する
    # 画像のサイズに応じてtick位置を計算
    rows, cols = size, size  # 行と列の数
    h, w = img.shape[:2]
    x_ticks = np.linspace(0, w, cols+1)[:-1] + w/(2*cols)  # 各セル中央
    y_ticks = np.linspace(0, h, rows+1)[:-1] + h/(2*rows)

    # 列ラベル（上）
    plt.xticks(x_ticks, [str(i+1) for i in range(cols)])
    # 行ラベル（左）
    plt.yticks(y_ticks, [str(i+1) for i in range(rows)])

    # 軸の上と右を有効化してラベルが上・左に来るように
    plt.tick_params(top=True, bottom=False, labeltop=True, labelbottom=False)
    plt.tick_params(left=True, right=False, labelleft=True, labelright=False)
    
    plt.xlabel("列", labelpad=20)
    plt.gca().xaxis.set_label_position('top')

    plt.ylabel("行", labelpad=20,rotation=0)
    plt.gca().yaxis.set_label_position('left')

    plt.grid(False)
    #plt.axis('off')
    plt.show()


                    #ユーザーによる数値の訂正
    while True:
        try:
            row = int(input("変更したい行番号 (1〜9) を入力 (終了する場合は0): "))
            if row == 0:  # 0を入力したら終了
                break

            col = int(input("変更したい列番号 (1〜9) を入力: "))
            new_value = int(input("新しい数字 (0〜9) を入力: "))

            # 入力チェック
            if 1 <= row <= 9 and 1 <= col <= 9 and 0 <= new_value <= 9:
                labels_grid[row-1, col-1] = new_value  # -1 して0始まりのインデックスに変換
            else:
                print("範囲外の値です")

        except ValueError:
            print("数字を入力してください")
    

    
    # 数独を解く
    size = 9
    numplace = NumberPlace.NumberPlace(size)
    numplace.setRules(labels_grid)

    if numplace.solve():
        for i in range(size):
            for j in range(size):
                if labels_grid[i][j] == 0:
                    pre.putAns(imgN, i, j, numplace.getCell(i, j), size)
    else:
        print("解けませんでした")

    # 元画像と解読結果を並べて表示
    plt.figure(figsize=(12, 6))

    plt.subplot(1, 2, 1)
    plt.title("Original")
    plt.imshow(cv2.cvtColor(num_img, cv2.COLOR_BGR2RGB))
    plt.axis('off')

    plt.subplot(1, 2, 2)
    plt.title("Solved")
    plt.imshow(cv2.cvtColor(imgN, cv2.COLOR_BGR2RGB))
    plt.axis('off')

    plt.show()
















